### Installing & Importing all the necessary packages

Update system packages, install `libgl1`, and install the `openpyxl` library for handling Excel files

- **`albumentations`**: Library for image augmentation to enhance training data diversity.
- **`sweetviz`**: Generates high-density visualizations of pandas DataFrames for quick data analysis.
- **`grad-cam`**: Visualizes important image regions for CNN predictions using Grad-CAM.
- **`lime`**: Provides local explanations for machine learning model predictions.
- **`pandas_profiling`**: Creates detailed reports of pandas DataFrames for exploratory data analysis.
- **`shap`**: Explains model predictions by attributing feature contributions using Shapley values.
- **`Keras-Preprocessing`**: Offers utilities for preprocessing data, including image and text transformations.

In [ ]:
from IPython.display import clear_output

!sudo apt-get update && apt-get install libgl1 -y
!pip install openpyxl -q
!pip install albumentations sweetviz grad-cam lime pandas_profiling shap Keras-Preprocessing -q

clear_output()

In [ ]:
# Standard Libraries
import os
import math
import shutil
import warnings
from typing import Dict, Optional

# Data Handling
import pandas as pd
import numpy as np
import cv2
from PIL import Image

# Plotting
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib.cm as cm
from matplotlib.colors import LinearSegmentedColormap

# Skimage
from skimage.segmentation import slic, mark_boundaries

# TensorFlow and Keras
import tensorflow as tf
from tensorflow.keras.preprocessing.image import load_img, img_to_array, ImageDataGenerator
from tensorflow.keras import Sequential, Model
from tensorflow.keras.layers import Input, Dense, BatchNormalization, Flatten, Conv2D, MaxPooling2D, Dropout, GlobalAveragePooling2D, LeakyReLU
from tensorflow.keras.optimizers import Adam, Adamax
from tensorflow.keras.applications import ResNet50V2, ResNet101V2, InceptionV3, InceptionResNetV2, MobileNetV2, DenseNet169, NASNetMobile, EfficientNetB7, ConvNeXtBase
from tensorflow.keras.callbacks import TensorBoard, ModelCheckpoint, EarlyStopping, ReduceLROnPlateau, CSVLogger, LearningRateScheduler

# Keras (Standalone)
import keras
from keras.models import Sequential, Model
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPooling2D, BatchNormalization
from keras.optimizers import Adam, SGD, Adagrad, Adadelta, RMSprop, Nadam
from keras.utils import to_categorical
from keras.callbacks import EarlyStopping, ModelCheckpoint, History, ReduceLROnPlateau, CSVLogger, LearningRateScheduler

# Machine Learning
from sklearn.model_selection import train_test_split, KFold, GridSearchCV
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import (accuracy_score, precision_score, recall_score, classification_report,
                             confusion_matrix, roc_curve, roc_auc_score, matthews_corrcoef, hamming_loss, f1_score, precision_recall_curve)

# Lime and Shap
from lime import lime_image
import shap

# PyTorch Grad-CAM
from pytorch_grad_cam import GradCAM

# Sweetviz (for EDA)
import sweetviz

# IPython for display
from IPython.display import Image, display

# Magic commands (for Jupyter Notebooks)
%matplotlib inline
%load_ext tensorboard
%reload_ext tensorboard

### Data Loading and Cleaning

Load class labels from an Excel file, binary test labels from a CSV, clean the class labels DataFrame by removing empty rows and columns, and display the cleaned data.

In [5]:
la1=pd.read_excel('/workspace/anushka saini/train_val/class_label.xlsx')
la1

,imagePath,Healthy
0,100image13.jpg,1
1,100image2.jpg,1
2,100image65.jpg,0
3,100image71.jpg,0
4,100image83.jpg,0
...,...,...
3195,pco_4.jpg,1
3196,pco_5.jpg,0
3197,pco_6.jpg,0
3198,pco_7.jpg,0


In [6]:
la2 = pd.read_csv('/workspace/anushka saini/test/test_label_binary.csv')
la2

,imagePath,Healthy
0,image10000.jpg,1
1,image10001.jpg,0
2,image10002.jpg,1
3,image10003.jpg,0
4,image10004.jpg,0
...,...,...
1463,image11463.jpg,0
1464,image11464.jpg,0
1465,image11465.jpg,0
1466,image11466.jpg,1


In [7]:
df = la1.dropna(how='all').dropna(how='all', axis=1)
print(df)

           imagePath  Healthy
0     100image13.jpg        1
1      100image2.jpg        1
2     100image65.jpg        0
3     100image71.jpg        0
4     100image83.jpg        0
...              ...      ...
3195       pco_4.jpg        1
3196       pco_5.jpg        0
3197       pco_6.jpg        0
3198       pco_7.jpg        0
3199       pco_8.jpg        0

[3200 rows x 2 columns]


### Train-Validate Split

Split the DataFrame into training and validation sets based on specified percentages, using optional random seed for reproducibility.

In [8]:
def train_validate_split(df, train_percent=.8, validate_percent=.2, seed=None):
    np.random.seed(seed)
    perm = np.random.permutation(df.index)
    m = len(df.index)
    train_end = int(train_percent * m)
    validate_end = int(validate_percent * m) + train_end
    train = df.iloc[perm[:train_end]]
    validate = df.iloc[perm[train_end:validate_end]]
    # test = df.iloc[perm[validate_end:]]
    return train, validate

In [9]:
train, validate = train_validate_split(df)
train

,imagePath,Healthy
3195,pco_4.jpg,1
2932,image3595.jpg,0
1377,image0973.jpg,0
1364,image0959.jpg,0
1468,image1075.jpg,0
...,...,...
2978,image3643.jpg,0
3061,image3733.jpg,0
1638,image1472.jpg,1
1484,image1092.jpg,0


In [10]:
validate

,imagePath,Healthy
3077,image3753.jpg,0
2465,image3062.jpg,0
355,49image0041.jpg,0
892,image0447.jpg,0
97,141image0125.jpg,1
...,...,...
938,image0500.jpg,0
318,2image50.jpg,0
482,90image15.jpg,0
715,image0257.jpg,0


### Image and Label Preparation

Map labels, load and preprocess images for training, validation, and testing, and display shapes of image arrays and labels.

In [11]:
label_mapping = {1: "Healthy", 0: "Unhealthy"}
df['Label'] = df['Healthy'].map(label_mapping)

In [12]:
train_labels = train[["Healthy"]].values
train_image_paths = [os.path.join('/workspace/anushka saini/train_val/images', filename) for filename in train['imagePath']] #to change when using GPU. similarly for validate and test
train_images = []
for train_image_path in train_image_paths:
    image = load_img(train_image_path, target_size=(300, 300)) # change this when you run on GPU. 320X320 rahegi
    image = img_to_array(image) / 255.0  # Normalize pixel values between 0 and 1
    train_images.append(image)
train_images = np.array(train_images, dtype=np.float32)

In [13]:
train_images.shape

(2560, 300, 300, 3)

In [14]:
train_labels.shape

(2560, 1)

In [15]:
validate_labels = validate[["Healthy"]].values
validate_image_paths = [os.path.join('/workspace/anushka saini/train_val/images', filename) for filename in validate['imagePath']]
validate_images = []
for validate_image_path in validate_image_paths:
    image = load_img(validate_image_path, target_size=(300, 300)) # change this when you run on GPU
    image = img_to_array(image) / 255.0  # Normalize pixel values between 0 and 1
    validate_images.append(image)
validate_images = np.array(validate_images, dtype=np.float32)

In [16]:
validate_images.shape

(640, 300, 300, 3)

In [17]:
validate_labels.shape

(640, 1)

In [18]:
test_labels = la2[["Healthy"]].values
test_image_paths = [os.path.join('/workspace/anushka saini/test/images', filename) for filename in la2['imagePath']]
test_images = []
for test_image_path in test_image_paths:
    image = load_img(test_image_path, target_size=(300, 300)) # change this when you run on GPU
    image = img_to_array(image) / 255.0  # Normalize pixel values between 0 and 1
    test_images.append(image)
test_images = np.array(test_images, dtype=np.float32)

In [19]:
test_images.shape

(1468, 300, 300, 3)

In [20]:
test_labels.shape

(1468, 1)

### Custom DenseNet169 Model for Binary Classification with Logging

Customizes and compiles a DenseNet169 model for binary classification with training logging.

In [21]:
base_model=DenseNet169(weights='imagenet', include_top=False,input_shape=(300,300,3))

2024-01-29 08:24:44.458537: I tensorflow/core/platform/cpu_feature_guard.cc:194] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE3 SSE4.1 SSE4.2 AVX
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-01-29 08:24:44.564423: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1621] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 38318 MB memory:  -> device: 0, name: NVIDIA A100-SXM4-40GB, pci bus id: 0000:81:00.0, compute capability: 8.0


51877672/51877672 [==============================] - 5s 0us/step


In [22]:
x = base_model.output
x = Flatten()(x)
predictions = Dense(1, activation='sigmoid',
    use_bias=True,
    kernel_initializer="glorot_uniform",
    bias_initializer="zeros",
    kernel_regularizer=None,
    bias_regularizer=None,
    activity_regularizer=None,
    kernel_constraint=None,
    bias_constraint=None,)(x)

model = Model(inputs=base_model.input, outputs=predictions)
for layer in base_model.layers:
    layer.trainable = False
model.compile(optimizer=Adam(), loss='binary_crossentropy', metrics=['accuracy', 'binary_accuracy', 'categorical_accuracy', 'top_k_categorical_accuracy', 'Precision', 'Recall'])
csv_logger = CSVLogger("DenseNet169_model_history_log.csv", append=True) #to be changed as per model

In [26]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 300, 300, 3  0           []                               
                                )]                                                                
                                                                                                  
 zero_padding2d (ZeroPadding2D)  (None, 306, 306, 3)  0          ['input_1[0][0]']                
                                                                                                  
 conv1/conv (Conv2D)            (None, 150, 150, 64  9408        ['zero_padding2d[0][0]']         
                                )                                                                 
                                                                                              

### Training, Saving, and Evaluating a DenseNet169 Model

Trains the DenseNet169 model, saves and reloads weights, and evaluates performance on validation and test datasets using classification reports.

In [28]:
model.fit(train_images,train_labels,validation_data=(validate_images,validate_labels),batch_size=16,
          epochs=250,callbacks=csv_logger,shuffle=False,class_weight=None,steps_per_epoch=None,
          validation_steps=None,
          validation_batch_size=None,
          validation_freq=1,
          max_queue_size=10,
          workers=1,
          use_multiprocessing=False) #make it 250

Epoch 1/250


2024-01-29 08:25:47.289300: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:428] Loaded cuDNN version 8700
2024-01-29 08:25:47.418255: I tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:648] TensorFloat-32 will be used for the matrix multiplication. This will only be logged once.


160/160 [==============================] - 14s 44ms/step - loss: 2.9538 - accuracy: 0.7063 - binary_accuracy: 0.7063 - categorical_accuracy: 1.0000 - top_k_categorical_accuracy: 1.0000 - precision: 0.4826 - recall: 0.4773 - val_loss: 2.2733 - val_accuracy: 0.7094 - val_binary_accuracy: 0.7094 - val_categorical_accuracy: 1.0000 - val_top_k_categorical_accuracy: 1.0000 - val_precision: 0.4790 - val_recall: 0.6477
Epoch 2/250
160/160 [==============================] - 4s 27ms/step - loss: 2.0600 - accuracy: 0.7477 - binary_accuracy: 0.7477 - categorical_accuracy: 1.0000 - top_k_categorical_accuracy: 1.0000 - precision: 0.5563 - recall: 0.5502 - val_loss: 2.7823 - val_accuracy: 0.6641 - val_binary_accuracy: 0.6641 - val_categorical_accuracy: 1.0000 - val_top_k_categorical_accuracy: 1.0000 - val_precision: 0.4325 - val_recall: 0.7102
Epoch 3/250
160/160 [==============================] - 4s 26ms/step - loss: 1.9616 - accuracy: 0.7777 - binary_accuracy: 0.7777 - categorical_accuracy: 1.0000 

In [29]:
model.save_weights('DenseNet169_weights.h5')
model.save('DenseNet169_model.h5')
model.load_weights('DenseNet169_weights.h5')

In [ ]:
validate_labels_pred = model.predict(validate_images)

# Apply threshold for binary classification
threshold = 0.5
validate_labels_pred_binary = (validate_labels_pred > threshold).astype(int)

# Extract binary labels
validate_labels = validate[["Healthy"]].values

# Generate classification report
print(classification_report(validate_labels, validate_labels_pred_binary))

20/20 [==============================] - 224s 11s/step
              precision    recall  f1-score   support

           0       0.88      0.98      0.93       460
           1       0.92      0.66      0.77       180

    accuracy                           0.89       640
   macro avg       0.90      0.82      0.85       640
weighted avg       0.89      0.89      0.88       640



In [ ]:
test_labels_pred = model.predict(test_images)

# Apply threshold for binary classification
threshold = 0.5
test_labels_pred_binary = (test_labels_pred > threshold).astype(int)

# Extract binary labels
test_labels = la2[["Healthy"]].values

# Generate classification report
print(classification_report(test_labels, test_labels_pred_binary))


46/46 [==============================] - 501s 11s/step
              precision    recall  f1-score   support

           0       0.85      0.96      0.90      1051
           1       0.87      0.57      0.69       417

    accuracy                           0.85      1468
   macro avg       0.86      0.77      0.80      1468
weighted avg       0.85      0.85      0.84      1468

